In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '2', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 1000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 10, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

In [8]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

# run model 

## init

In [9]:
def get_feed_dict(batch, mode='train'):
    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    keep_prob = config.keep_prob if mode == 'train' else 1.0
    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow])
t_variables['keep_prob'] = tf.placeholder(tf.float32)

## build model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

## define loss

In [12]:
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))

global_step = tf.Variable(0, name='global_step',trainable=False)

loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)

grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]
opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

In [13]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, ppl_mean

def print_topic_sample():
    topics_freq_bow_idxs = bow_idxs[sess.run(topics_freq_bow_indices)]
    for i, topic_freq_bow_idxs in enumerate(topics_freq_bow_idxs):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))

In [14]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','VALID:','TM','','',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG']]))))

In [15]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log = sess.run(tf.train.get_global_step())
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            
            # visualize topic
            print_topic_sample()

            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                      VALID:    TM          \
       Time  Ep    Ct    LOSS   PPL     NLL    KL   REG    LOSS   PPL     NLL   
1         0   0     0  124.32  1036  122.99  0.43  0.90  117.20  1032  115.99   
1001      5   0  1000  113.72   548  112.86  0.58  0.28  104.82   500  103.90   
2001      5   0  2000  112.69   522  111.68  0.84  0.18  104.47   480  103.36   
2276      1   1     0  112.47   517  111.43  0.89  0.16  104.16   476  103.11   
3276      5   1  1000  112.01   504  110.86  1.03  0.13  103.98   463  102.75   
4276      5   1  2000  111.79   493  110.54  1.14  0.11  103.69   452  102.31   
4551      1   2     0  111.78   491  110.50  1.17  0.10  103.53   442  102.02   
5551      5   2  1000  111.63   482  110.27  1.26  0.09  103.53   439  101.92   
6551      5   2  2000  111.45   476  110.02  1.34  0.09  103.50   443  101.94   
6826      1   3     0  111.37   474  109.93  1.35  0.08  103.23   429  101.47   
7826      5   3  1000  111.17   468  109.67  1.42  0.08  103.29   430  101.53   
8826      5   3  2000  111.10   464  109.55  1.48  0.07  103.17   426  101.42   
9101      1   4     0  111.09   463  109.53  1.49  0.07  103.14   427  101.43   
10101     4   4  1000  111.01   459  109.40  1.53  0.07  103.08   424  101.31   
11101     5   4  2000  110.91   456  109.27  1.57  0.07  103.05   421  101.18   
11376     1   5     0  110.89   455  109.24  1.58  0.07  102.89   413  100.97   
12376     4   5  1000  110.77   452  109.09  1.62  0.06  103.10   423  101.29   
13376     5   5  2000  110.74   450  109.03  1.65  0.06  102.99   418  101.09   
13651     1   6     0  110.73   449  109.01  1.66  0.06  103.02   417  101.09   
14651     5   6  1000  110.67   447  108.93  1.69  0.06  102.95   418  101.10   
15651     5   6  2000  110.62   445  108.85  1.71  0.06  102.99   415  101.01   
15926     1   7     0  110.61   444  108.83  1.72  0.06  102.95   412  100.86   
16926     5   7  1000  110.58   442  108.78  1.75  0.06  102.93   415  100.89   
17926     5   7  2000  110.51   441  108.69  1.77  0.06  102.83   410  100.85   
18201     1   8     0  110.51   440  108.68  1.78  0.06  102.79   412  100.83   
19201     5   8  1000  110.46   439  108.61  1.80  0.05  102.93   411  100.92   
20201     5   8  2000  110.44   437  108.58  1.81  0.05  102.85   414  100.85   
20476     1   9     0  110.43   437  108.56  1.82  0.05  102.84   412  100.86   
21476     4   9  1000  110.39   436  108.51  1.84  0.05  102.91   415  100.89   
22476     5   9  2000  110.36   434  108.45  1.85  0.05  102.78   408  100.68   
...     ...  ..   ...     ...   ...     ...   ...   ...     ...   ...     ...   
91001     1  40     0  109.66   408  107.40  2.22  0.04  102.61   399  100.33   
92001     4  40  1000  109.66   407  107.39  2.23  0.04  102.49   395  100.18   
93001     4  40  2000  109.65   407  107.38  2.23  0.04  102.40   394  100.15   
93276     1  41     0  109.65   407  107.38  2.23  0.04  102.73   404  100.44   
94276     4  41  1000  109.65   407  107.38  2.23  0.04  102.49   395  100.16   
95276     4  41  2000  109.64   407  107.37  2.23  0.04  102.50   396  100.20   
95551     1  42     0  109.65   407  107.37  2.23  0.04  102.50   396  100.16   
96551     4  42  1000  109.64   407  107.36  2.23  0.04  102.60   398  100.30   
97551     4  42  2000  109.63   407  107.35  2.24  0.04  102.41   393  100.12   
97826     1  43     0  109.64   407  107.36  2.24  0.04  102.52   396  100.25   
98826     4  43  1000  109.63   406  107.35  2.24  0.04  102.36   392  100.07   
99826     4  43  2000  109.63   406  107.34  2.24  0.04  102.49   394  100.19   
100101    1  44     0  109.63   406  107.34  2.24  0.04  102.58   396  100.27   
101101    4  44  1000  109.63   406  107.34  2.24  0.04  102.51   394  100.20   
102101    4  44  2000  109.62   406  107.33  2.25  0.04  102.42   393  100.14   
102376    1  45     0  109.62   406  107.33  2.25  0.04  102.56   397  100.25   
103376    4  45  1000  109.61   406  107.32  2.25  0.0

0  BOW: ! love perfect price bought color recommend highly loves quality
1  BOW: ... quality $ price bought amazon item reviews return buy
2  BOW: sleeve inch inside zipper smell padding - protection bit neoprene
3  BOW: cover color keyboard ! love apple mac easy hard pro
4  BOW: bottom top plastic cover part back speck months feet piece
5  BOW: pocket room small power ipad mouse charger inside carry cord
6  BOW: strap zipper handle months shoulder - zippers straps 've broke
7  BOW: & ; sleeve pro big made size bit 'm -
8  BOW: carry pockets room space compartments comfortable back work plenty lot
9  BOW: pro air protect protection perfectly mac soft nice hard recommend


# write coherence file

In [16]:
topics_freq_bow_idxs = bow_idxs[sess.run(topics_freq_bow_indices)]
topics_freq_tokens = [' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]) for topic_freq_bow_idxs in topics_freq_bow_idxs]

In [17]:
path_coherence = 'npmi/data/bags/gsm.txt'
with open(path_coherence, 'w') as f:
    f.write('\n'.join(topics_freq_tokens))

# confirm variables

In [16]:
debug_value([topic_losses_recon, n_bow])

Neg:0 : [ 62.297527  56.136356 154.29373   88.97808  102.96145   87.09083
 227.5929    97.16991   73.68268   71.77301   31.193447 142.24402
  72.797424  62.079376  71.74945  145.35117   87.5488   117.33721
  27.166435  64.628746  44.766636  56.912777  66.08688   85.55026
 105.20309  104.787796  80.63867   57.57313   91.79883   82.65097
  84.80786   67.81983   29.320303 176.21875   38.674168 275.64484
  46.82713  171.4018    71.25316   69.34422   54.380184  85.18074
 102.59453   85.81177  126.17367   93.92539  120.23218   98.609024
  70.81109  238.25818  119.59937  179.05786  139.86867 ]
Sum_2:0 : [10. 10. 26. 14. 15. 14. 39. 17. 13. 12.  5. 24. 13. 11. 12. 23. 14. 19.
  5. 12.  7.  9. 11. 15. 16. 19. 14.  9. 16. 14. 14. 12.  5. 29.  7. 44.
  9. 28. 12. 12. 10. 14. 17. 14. 21. 17. 19. 17. 12. 38. 19. 27. 25.]


In [17]:
debug_value([tf.exp(-tf.divide(topic_losses_recon, n_bow))])

Exp_1:0 : [0.00177701 0.00393379 0.00284705 0.00204366 0.00117886 0.00261166
 0.00293802 0.00326275 0.00467356 0.00429009 0.00235161 0.00262558
 0.00342418 0.00388673 0.00315612 0.0017324  0.00197161 0.00368242
 0.0028409  0.00355141 0.00157192 0.001876   0.00237097 0.0025489
 0.00120646 0.00365071 0.00304019 0.00170096 0.00249563 0.00308826
 0.0036157  0.0036778  0.00452899 0.0020654  0.00542014 0.00180567
 0.00704606 0.00215958 0.00298581 0.00290098 0.00407997 0.00248433
 0.00203083 0.00226861 0.00196002 0.00388822 0.00168553 0.0027972
 0.00260488 0.00200653 0.00182105 0.00125839 0.00407746]


### test

In [18]:
debug_shape([bow, hidden_bow, latents_bow, prob_topic, bow_embeddings, topic_embeddings, topic_bow, prob_bow])

NameError: name 'bow' is not defined

In [ ]:
debug_shape([topic_losses_recon, topic_loss_recon, n_bow, ppls, topic_embeddings_norm, tf.expand_dims(topic_angles_mean, -1), topic_angles_vars])

In [ ]:
debug_value([tf.reduce_sum(tf.square(topic_embeddings_norm), 1)], return_value=True)[0]

In [ ]:
debug_value([tf.reduce_sum(prob_topic, -1), tf.reduce_sum(topic_bow, -1), tf.reduce_sum(tf.exp(prob_bow), 1)])

In [ ]:
sigma_bow = tf.exp(0.5 * logvars_bow)
dist_bow = tfd.Normal(means_bow, sigma_bow)
dist_std = tfd.Normal(0., 1.)
topic_loss_kl_tmp = tf.reduce_mean(tf.reduce_sum(tfd.kl_divergence(dist_bow, dist_std), 1))

In [ ]:
debug_value([topic_loss_recon, topic_loss_kl, topic_loss_kl_tmp])

In [ ]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [ ]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

In [ ]:
_output_logits

In [ ]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


In [ ]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

In [ ]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

In [ ]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [ ]:
_idxs = _dec_target_idxs_do

In [ ]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [ ]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

In [ ]:
_recon_loss

In [ ]:
_kl_losses.shape